In [ ]:
from sklearn.metrics import (
    confusion_matrix,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    classification_report
)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import DenseNet121
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


X_test = np.load("/content/drive/MyDrive/AI/x_test1.npy")
y_test = np.load("/content/drive/MyDrive/AI/y_test1.npy")


print(f'Shape of X_test: {X_test.shape}')
print(f'Shape of y_test: {y_test.shape}')


densenet = DenseNet121(
        weights='imagenet',
        include_top=False,
        input_shape=(256,256,3)
)
resnet = ResNet50(weights='imagenet',
                   include_top=False,
                     input_shape=(256, 256, 3))
vgg19 = VGG19(weights='imagenet',
               include_top=False,
                 input_shape=(256, 256, 3))


model = Sequential()
#model.add(densenet)
#model.add(resnet)
model.add(vgg19)

model.add(GlobalAveragePooling2D())
model.add(Dropout(0.5))
model.add(Dense(1024, activation='relu'))
model.add(Dense(5, activation='sigmoid'))  # Use sigmoid for multi-label classification

# Build the model explicitly
model.build((None, 256, 256, 3))  # Use the same shape as input_shape

# Compile the model (compilation done once before the loop)
model.compile(
    loss='binary_crossentropy',  # Use binary cross-entropy for multi-label
    optimizer=Adam(learning_rate=0.00005),
    metrics=['accuracy']
)

# Loop through each model weights file


for i in range(1, 12):

    # Load model weights
    weight_file_path = f"/content/model_weights{i}.weights.h5"
    try:
        model.load_weights(weight_file_path)
        print(f'Successfully loaded weights from {weight_file_path}')
    except Exception as e:
        print(f'Error loading weights from {weight_file_path}: {e}')
        continue  # Skip this iteration if weights can't be loaded

    # Predict
    y_pred = model.predict(X_test)
    y_pred_classes = (y_pred > 0.5).astype(int)

    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred_classes)
    precision = precision_score(y_test, y_pred_classes, average='weighted')
    recall = recall_score(y_test, y_pred_classes, average='weighted')
    f1 = f1_score(y_test, y_pred_classes, average='weighted')
    report = classification_report(y_test, y_pred_classes)

    # Print metrics
    print(f'---- Model Weights {i} ----')
    print(f'Accuracy: {accuracy}')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print(f'F1 Score: {f1}')
    print('Classification Report:')
    print(report)
    print('\n')

    # Plot confusion matrix
    # For multi-label, calculate confusion matrix for each label separately
    for idx in range(y_test.shape[1]):
        conf_matrix = confusion_matrix(y_test[:, idx], y_pred_classes[:, idx])
        plt.figure(figsize=(5, 4))
        sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
        plt.title(f'- Confusion Matrix for Label {idx} -')
        plt.xlabel('Predicted')
        plt.ylabel('Actual')
        plt.show()